## Kombinasi 7 :
- Delete duplicate
- Impute missing value using median/mode
- Outlier handling with winsorization
- Encoding
- Standard Scaler
- Feature selection with Decision Tree

In [139]:
# import library
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [140]:
df = pd.read_csv("../UFC_train.csv")
df_test = pd.read_csv('../regression/New_UFC_Test_Regression_X(3).csv')
df.head()

,R_fighter,B_fighter,Referee,date,location,title_bout,weight_class,B_avg_KD,B_avg_opp_KD,B_avg_SIG_STR_pct,...,R_win_by_KO/TKO,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage,R_Stance,R_Height_cms,R_Reach_cms,R_Weight_lbs,B_age,R_age,Winner
0,Joe Riggs,Joe Doerksen,Steve Mazzagatti,2004-08-21,"Las Vegas, Nevada, USA",False,Middleweight,NaN,NaN,NaN,...,0,0,0,Southpaw,182.88,177.80,185.0,26.0,21.0,Red
1,Jorge Masvidal,Al Iaquinta,Keith Peterson,2015-04-04,"Fairfax, Virginia, USA",False,Lightweight,1.156250,0.000000,0.394141,...,0,1,0,Orthodox,180.34,187.96,170.0,27.0,30.0,Blue
2,Dan Stittgen,Stephen Thompson,Josh Rosenthal,2012-02-04,"Las Vegas, Nevada, USA",False,Welterweight,NaN,NaN,NaN,...,0,0,0,Orthodox,185.42,NaN,170.0,28.0,31.0,Blue
3,Josh Koscheck,Johny Hendricks,Kevin Mulhall,2012-05-05,"East Rutherford, New Jersey, USA",False,Welterweight,0.695312,0.000000,0.783359,...,6,3,0,Orthodox,177.80,185.42,170.0,28.0,34.0,Blue
4,John Dodson,Manvel Gamburyan,James Warring,2016-04-16,"Tampa, Florida, USA",False,Bantamweight,0.500000,0.266602,0.381462,...,3,0,1,Orthodox,160.02,167.64,135.0,34.0,31.0,Red


In [141]:
# Delete duplicate
print(f"Shape before dropping duplicate : {df.shape}")
df.drop_duplicates(inplace=True)
print(f"Shape after dropping duplicate : {df.shape}")

Shape before dropping duplicate : (5410, 144)
Shape after dropping duplicate : (5410, 144)


In [142]:
def check_null(df):
    col_na = df.isnull().sum().sort_values(ascending=True)
    percent = col_na / len(df)
    missing_data = pd.concat([col_na, percent], axis=1, keys=['Total', 'Percent'])

    if (missing_data[missing_data['Total'] > 0].shape[0] == 0):
        print("Tidak ditemukan missing value pada dataset")
    else:
        print(missing_data[missing_data['Total'] > 0])

In [143]:
check_null(df)

                         Total   Percent
R_Weight_lbs                 2  0.000370
R_Height_cms                 4  0.000739
B_Weight_lbs                 8  0.001479
B_Height_cms                10  0.001848
R_Stance                    27  0.004991
...                        ...       ...
B_avg_GROUND_landed       1293  0.239002
B_avg_GROUND_att          1293  0.239002
B_avg_opp_CLINCH_landed   1293  0.239002
B_avg_TD_att              1293  0.239002
B_avg_HEAD_landed         1293  0.239002

[109 rows x 2 columns]


In [144]:
# Split string and number
# Split dataframe into object type and number type
df_object = df.select_dtypes(include='object')
df_number = df.select_dtypes(include=np.number)

df_test_object = df_test.select_dtypes(include='object')
df_test_number = df_test.select_dtypes(include=np.number)

In [145]:
# Impute missing value using median/mode
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy='median')
df_imputed_number = imputer.fit_transform(df_number)
df_imputed_number = pd.DataFrame(df_imputed_number, columns=df_number.columns)
df_test_imputed_number = imputer.fit_transform(df_test_number)
df_test_imputed_number = pd.DataFrame(df_test_imputed_number, columns=df_test_number.columns)

imputer = SimpleImputer(strategy='most_frequent')
df_imputed_object = imputer.fit_transform(df_object)
df_imputed_object = pd.DataFrame(df_imputed_object, columns=df_object.columns)
df_test_imputed_object = imputer.fit_transform(df_test_object)
df_test_imputed_object = pd.DataFrame(df_test_imputed_object, columns=df_test_object.columns)

In [146]:
# Drop R_fighter, B_fighter, Referee, location, date
df_object = df_object.drop(['R_fighter', 'B_fighter', 'Referee', 'location', 'date', 'Winner'], axis=1)
df_test_object = df_test_object.drop(['R_fighter', 'B_fighter', 'Referee', 'location', 'date'], axis=1)

In [147]:
# Combine df_object and df_imputed_number
df = pd.concat([df_object, df_imputed_number], axis=1)
df_test = pd.concat([df_test_object, df_test_imputed_number], axis=1)
df.head()

,weight_class,B_Stance,R_Stance,B_avg_KD,B_avg_opp_KD,B_avg_SIG_STR_pct,B_avg_opp_SIG_STR_pct,B_avg_TD_pct,B_avg_opp_TD_pct,B_avg_SUB_ATT,...,R_win_by_Decision_Split,R_win_by_Decision_Unanimous,R_win_by_KO/TKO,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage,R_Height_cms,R_Reach_cms,R_Weight_lbs,B_age,R_age
0,Middleweight,Orthodox,Southpaw,0.015625,0.000000,0.450000,0.427500,0.250000,0.200000,0.15625,...,0.0,0.0,0.0,0.0,0.0,182.88,177.80,185.0,26.0,21.0
1,Lightweight,Orthodox,Orthodox,1.156250,0.000000,0.394141,0.352422,0.239219,0.011484,0.15625,...,0.0,4.0,0.0,1.0,0.0,180.34,187.96,170.0,27.0,30.0
2,Welterweight,Orthodox,Orthodox,0.015625,0.000000,0.450000,0.427500,0.250000,0.200000,0.15625,...,0.0,0.0,0.0,0.0,0.0,185.42,182.88,170.0,28.0,31.0
3,Welterweight,Southpaw,Orthodox,0.695312,0.000000,0.783359,0.185547,0.088281,0.104375,0.09375,...,1.0,5.0,6.0,3.0,0.0,177.80,185.42,170.0,28.0,34.0
4,Bantamweight,Orthodox,Orthodox,0.500000,0.266602,0.381462,0.456558,0.429614,0.469570,1.37793,...,0.0,2.0,3.0,0.0,1.0,160.02,167.64,135.0,34.0,31.0


In [148]:
check_null(df)

          Total   Percent
R_Stance     27  0.004991
B_Stance     64  0.011830


In [149]:
# Impute missing R_stance and B_stance with mode
df['R_Stance'].fillna(df['R_Stance'].mode()[0], inplace=True)
df['B_Stance'].fillna(df['B_Stance'].mode()[0], inplace=True)

df_test['R_Stance'].fillna(df_test['R_Stance'].mode()[0], inplace=True)
df_test['B_Stance'].fillna(df_test['B_Stance'].mode()[0], inplace=True)

In [150]:
check_null(df)

Tidak ditemukan missing value pada dataset


In [151]:
def check_outlier(df):
    Q1 = df.quantile(0.25)
    Q3 = df.quantile(0.75)

    # Menghitung RUB dan RLB.
    IQR = Q3 - Q1
    lower_limit = Q1 - 1.5*IQR
    upper_limit = Q3 + 1.5*IQR

    # Menampilkan banyaknya outlier pada atribut.
    outliers = (df < lower_limit) | (df > upper_limit)
    print ("Outlier pada tiap atribut:")
    print(outliers.sum())

    return outliers

In [152]:
# Split string and number
# Split dataframe into object type and number type
df_object = df.select_dtypes(include='object')
df_number = df.select_dtypes(include=np.number)

df_test_object = df_test.select_dtypes(include='object')
df_test_number = df_test.select_dtypes(include=np.number)

In [153]:
outliers = dict(check_outlier(df_number).sum())
print("\n\npercentage of outliers in each column:")
for key in outliers.keys():
    print(f"{key} = {outliers[key]/df_number[key].shape[0] * 100}%")

Outlier pada tiap atribut:
B_avg_KD                  561
B_avg_opp_KD             1142
B_avg_SIG_STR_pct         448
B_avg_opp_SIG_STR_pct     425
B_avg_TD_pct              186
                         ... 
R_Height_cms               19
R_Reach_cms                76
R_Weight_lbs              301
B_age                      31
R_age                      74
Length: 134, dtype: int64


percentage of outliers in each column:
B_avg_KD = 10.369685767097966%
B_avg_opp_KD = 21.109057301293902%
B_avg_SIG_STR_pct = 8.280961182994455%
B_avg_opp_SIG_STR_pct = 7.8558225508317925%
B_avg_TD_pct = 3.4380776340110906%
B_avg_opp_TD_pct = 6.210720887245841%
B_avg_SUB_ATT = 8.22550831792976%
B_avg_opp_SUB_ATT = 5.970425138632162%
B_avg_REV = 21.293900184842883%
B_avg_opp_REV = 23.696857670979668%
B_avg_SIG_STR_att = 6.358595194085027%
B_avg_SIG_STR_landed = 5.582255083179297%
B_avg_opp_SIG_STR_att = 6.22920517560074%
B_avg_opp_SIG_STR_landed = 6.598890942698706%
B_avg_TOTAL_STR_att = 4.787430683918669%
B_a

In [154]:
from scipy.stats.mstats import winsorize

# Identify columns with more than 0.5% outliers
outlier_columns = []
for key in outliers.keys():
    if outliers[key] / df_number[key].shape[0] > 0.005:
        outlier_columns.append(key)

# Winsorize the identified columns
for column in outlier_columns:
    df_number[column] = winsorize(df_number[column], limits=(0, 0.1))

# Combine df_object and df_number
df = pd.concat([df_object, df_number], axis=1)
df.head()


,weight_class,B_Stance,R_Stance,B_avg_KD,B_avg_opp_KD,B_avg_SIG_STR_pct,B_avg_opp_SIG_STR_pct,B_avg_TD_pct,B_avg_opp_TD_pct,B_avg_SUB_ATT,...,R_win_by_Decision_Split,R_win_by_Decision_Unanimous,R_win_by_KO/TKO,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage,R_Height_cms,R_Reach_cms,R_Weight_lbs,B_age,R_age
0,Middleweight,Orthodox,Southpaw,0.015625,0.000000,0.450000,0.427500,0.250000,0.200000,0.15625,...,0.0,0.0,0.0,0.0,0.0,182.88,177.80,185.0,26.0,21.0
1,Lightweight,Orthodox,Orthodox,0.649799,0.000000,0.394141,0.352422,0.239219,0.011484,0.15625,...,0.0,4.0,0.0,1.0,0.0,180.34,187.96,170.0,27.0,30.0
2,Welterweight,Orthodox,Orthodox,0.015625,0.000000,0.450000,0.427500,0.250000,0.200000,0.15625,...,0.0,0.0,0.0,0.0,0.0,185.42,182.88,170.0,28.0,31.0
3,Welterweight,Southpaw,Orthodox,0.649799,0.000000,0.586479,0.185547,0.088281,0.104375,0.09375,...,1.0,4.0,4.0,2.0,0.0,177.80,185.42,170.0,28.0,34.0
4,Bantamweight,Orthodox,Orthodox,0.500000,0.266602,0.381462,0.456558,0.429614,0.469570,1.06250,...,0.0,2.0,3.0,0.0,0.0,160.02,167.64,135.0,34.0,31.0


In [155]:
outliers = dict(check_outlier(df_number).sum())
print("\n\npercentage of outliers in each column:")
for key in outliers.keys():
    print(f"{key} = {outliers[key]/df_number[key].shape[0] * 100}%")

c:\Python312\Lib\site-packages\numpy\lib\function_base.py:4823: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedArray.
  arr.partition(
c:\Python312\Lib\site-packages\numpy\lib\function_base.py:4823: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedArray.
  arr.partition(


Outlier pada tiap atribut:
B_avg_KD                  561
B_avg_opp_KD             1142
B_avg_SIG_STR_pct         202
B_avg_opp_SIG_STR_pct     158
B_avg_TD_pct                0
                         ... 
R_Height_cms               19
R_Reach_cms                35
R_Weight_lbs                0
B_age                       0
R_age                       4
Length: 134, dtype: int64


percentage of outliers in each column:
B_avg_KD = 10.369685767097966%
B_avg_opp_KD = 21.109057301293902%
B_avg_SIG_STR_pct = 3.7338262476894637%
B_avg_opp_SIG_STR_pct = 2.9205175600739373%
B_avg_TD_pct = 0.0%
B_avg_opp_TD_pct = 0.0%
B_avg_SUB_ATT = 0.0%
B_avg_opp_SUB_ATT = 0.0%
B_avg_REV = 21.293900184842883%
B_avg_opp_REV = 23.696857670979668%
B_avg_SIG_STR_att = 0.0%
B_avg_SIG_STR_landed = 0.0%
B_avg_opp_SIG_STR_att = 0.0%
B_avg_opp_SIG_STR_landed = 0.0%
B_avg_TOTAL_STR_att = 0.0%
B_avg_TOTAL_STR_landed = 0.0%
B_avg_opp_TOTAL_STR_att = 0.0%
B_avg_opp_TOTAL_STR_landed = 0.0%
B_avg_TD_att = 0.0%
B_avg_TD_lan

In [156]:
df['gender'] = df['weight_class'].apply(lambda x: 'women' if 'women' in x.lower() else 'male')
df_test['gender'] = df_test['weight_class'].apply(lambda x: 'women' if 'women' in x.lower() else 'male')

In [157]:
df.head()

,weight_class,B_Stance,R_Stance,B_avg_KD,B_avg_opp_KD,B_avg_SIG_STR_pct,B_avg_opp_SIG_STR_pct,B_avg_TD_pct,B_avg_opp_TD_pct,B_avg_SUB_ATT,...,R_win_by_Decision_Unanimous,R_win_by_KO/TKO,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage,R_Height_cms,R_Reach_cms,R_Weight_lbs,B_age,R_age,gender
0,Middleweight,Orthodox,Southpaw,0.015625,0.000000,0.450000,0.427500,0.250000,0.200000,0.15625,...,0.0,0.0,0.0,0.0,182.88,177.80,185.0,26.0,21.0,male
1,Lightweight,Orthodox,Orthodox,0.649799,0.000000,0.394141,0.352422,0.239219,0.011484,0.15625,...,4.0,0.0,1.0,0.0,180.34,187.96,170.0,27.0,30.0,male
2,Welterweight,Orthodox,Orthodox,0.015625,0.000000,0.450000,0.427500,0.250000,0.200000,0.15625,...,0.0,0.0,0.0,0.0,185.42,182.88,170.0,28.0,31.0,male
3,Welterweight,Southpaw,Orthodox,0.649799,0.000000,0.586479,0.185547,0.088281,0.104375,0.09375,...,4.0,4.0,2.0,0.0,177.80,185.42,170.0,28.0,34.0,male
4,Bantamweight,Orthodox,Orthodox,0.500000,0.266602,0.381462,0.456558,0.429614,0.469570,1.06250,...,2.0,3.0,0.0,0.0,160.02,167.64,135.0,34.0,31.0,male


In [158]:
df['weight_class'] = df['weight_class'].str.replace('Women', '')
df_test['weight_class'] = df_test['weight_class'].str.replace('Women', '')

In [159]:
# Get all weight_class values
weight_class = df['weight_class'].unique()
weight_class

array(['Middleweight', 'Lightweight', 'Welterweight', 'Bantamweight',
       'Flyweight', 'LightHeavyweight', 'Strawweight', 'Featherweight',
       'OpenWeight', 'Heavyweight', 'CatchWeight'], dtype=object)

In [160]:
# Create a dictionary to map weight_class values to numbers
weight_class_dict = {
    'CatchWeight' : 0,
    'Strawweight' : 1,
    'Flyweight' : 2,
    'Bantamweight' : 3,
    'Featherweight' : 4,
    'Lightweight' : 5,
    'Welterweight' : 6,
    'Middleweight' : 7,
    'LightHeavyweight' : 8,
    'Heavyweight' : 9,
    'OpenWeight' : 10,
}

gender_dict = {
    'male' : 1,
    'women' : 0
}

In [161]:
# Map each weight_class value to the correct number
df['weight_class'] = df['weight_class'].map(weight_class_dict)
df_test['weight_class'] = df_test['weight_class'].map(weight_class_dict)

In [162]:
# Map each gender value to the correct number
df['gender'] = df['gender'].map(gender_dict)
df_test['gender'] = df_test['gender'].map(gender_dict)

In [163]:
df.head()

,weight_class,B_Stance,R_Stance,B_avg_KD,B_avg_opp_KD,B_avg_SIG_STR_pct,B_avg_opp_SIG_STR_pct,B_avg_TD_pct,B_avg_opp_TD_pct,B_avg_SUB_ATT,...,R_win_by_Decision_Unanimous,R_win_by_KO/TKO,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage,R_Height_cms,R_Reach_cms,R_Weight_lbs,B_age,R_age,gender
0,7,Orthodox,Southpaw,0.015625,0.000000,0.450000,0.427500,0.250000,0.200000,0.15625,...,0.0,0.0,0.0,0.0,182.88,177.80,185.0,26.0,21.0,1
1,5,Orthodox,Orthodox,0.649799,0.000000,0.394141,0.352422,0.239219,0.011484,0.15625,...,4.0,0.0,1.0,0.0,180.34,187.96,170.0,27.0,30.0,1
2,6,Orthodox,Orthodox,0.015625,0.000000,0.450000,0.427500,0.250000,0.200000,0.15625,...,0.0,0.0,0.0,0.0,185.42,182.88,170.0,28.0,31.0,1
3,6,Southpaw,Orthodox,0.649799,0.000000,0.586479,0.185547,0.088281,0.104375,0.09375,...,4.0,4.0,2.0,0.0,177.80,185.42,170.0,28.0,34.0,1
4,3,Orthodox,Orthodox,0.500000,0.266602,0.381462,0.456558,0.429614,0.469570,1.06250,...,2.0,3.0,0.0,0.0,160.02,167.64,135.0,34.0,31.0,1


In [164]:
# Get the remaining object columns
df_object = df.select_dtypes(include='object')

# Perform one-hot encoding
df_encoded = pd.get_dummies(df_object)

# Combine the encoded columns with the original dataframe
df = pd.concat([df, df_encoded], axis=1)

# Drop the original object columns
df.drop(df_object.columns, axis=1, inplace=True)

df.head()


,weight_class,B_avg_KD,B_avg_opp_KD,B_avg_SIG_STR_pct,B_avg_opp_SIG_STR_pct,B_avg_TD_pct,B_avg_opp_TD_pct,B_avg_SUB_ATT,B_avg_opp_SUB_ATT,B_avg_REV,...,B_Stance_Open Stance,B_Stance_Orthodox,B_Stance_Sideways,B_Stance_Southpaw,B_Stance_Switch,R_Stance_Open Stance,R_Stance_Orthodox,R_Stance_Sideways,R_Stance_Southpaw,R_Stance_Switch
0,7,0.015625,0.000000,0.450000,0.427500,0.250000,0.200000,0.15625,0.125000,0.000,...,False,True,False,False,False,False,False,False,True,False
1,5,0.649799,0.000000,0.394141,0.352422,0.239219,0.011484,0.15625,0.132812,0.000,...,False,True,False,False,False,False,True,False,False,False
2,6,0.015625,0.000000,0.450000,0.427500,0.250000,0.200000,0.15625,0.125000,0.000,...,False,True,False,False,False,False,True,False,False,False
3,6,0.649799,0.000000,0.586479,0.185547,0.088281,0.104375,0.09375,0.062500,0.000,...,False,False,False,True,False,False,True,False,False,False
4,3,0.500000,0.266602,0.381462,0.456558,0.429614,0.469570,1.06250,0.000244,0.125,...,False,True,False,False,False,False,True,False,False,False


In [165]:
# Get the remaining object columns
df_test_object = df_test.select_dtypes(include='object')

# Perform one-hot encoding
df_test_encoded = pd.get_dummies(df_test_object)

# Combine the encoded columns with the original dataframe
df_test = pd.concat([df_test, df_test_encoded], axis=1)

# Drop the original object columns
df_test.drop(df_test_object.columns, axis=1, inplace=True)

df_test.head()


,weight_class,id,B_avg_KD,B_avg_opp_KD,B_avg_SIG_STR_pct,B_avg_opp_SIG_STR_pct,B_avg_TD_pct,B_avg_opp_TD_pct,B_avg_SUB_ATT,B_avg_opp_SUB_ATT,...,B_age,R_age,gender,B_Stance_Orthodox,B_Stance_Southpaw,B_Stance_Switch,R_Stance_Open Stance,R_Stance_Orthodox,R_Stance_Southpaw,R_Stance_Switch
0,1,0.0,0.500000,0.000000,0.530000,0.425000,0.000000,0.160000,0.000000,0.0000,...,30.0,23.0,0,True,False,False,False,True,False,False
1,3,1.0,0.000000,0.000000,0.565000,0.435000,0.750000,1.000000,0.500000,0.5000,...,30.0,27.0,0,True,False,False,False,True,False,False
2,7,2.0,0.568909,0.437805,0.583355,0.439665,0.004447,0.166195,0.004333,0.2547,...,45.0,38.0,1,True,False,False,False,False,True,False
3,2,3.0,0.500000,0.000000,0.435000,0.355000,0.250000,0.250000,2.000000,0.0000,...,26.0,29.0,1,False,True,False,False,True,False,False
4,3,4.0,0.000000,0.000000,0.150000,0.440000,0.330000,0.180000,0.000000,0.0000,...,27.0,27.0,1,True,False,False,False,False,True,False


In [166]:
# Export to CSV
df.to_csv('../Without Feature Selection/UFC_kombinasi7_wt_featureselection.csv', index=False)

In [167]:
from sklearn.tree import DecisionTreeRegressor

# Feature selection with Decision Tree
X = df.drop('B_Reach_cms', axis=1)
y = df['B_Reach_cms']

# Splitting data into training and testing data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2020)

model = DecisionTreeRegressor()
model.fit(X_train, y_train)

DecisionTreeRegressor()

In [168]:
feature_importances = model.feature_importances_

In [169]:
top_n = 20
top_features_indices = feature_importances.argsort()[-top_n:][::-1]

In [170]:
top_features_indices

array([ 64,  65, 132,  50, 130,   0,  54, 133, 129, 131, 112,  48,  47,
        32,  52, 101, 136,  70,   3,  94], dtype=int64)

In [171]:
# Get the names of the top features
top_features = X.columns[top_features_indices]
top_features

Index(['B_Height_cms', 'B_Weight_lbs', 'B_age', 'B_total_rounds_fought',
       'R_Reach_cms', 'weight_class', 'B_longest_win_streak', 'R_age',
       'R_Height_cms', 'R_Weight_lbs', 'R_avg_CTRL_time(seconds)',
       'B_avg_opp_CTRL_time(seconds)', 'B_avg_CTRL_time(seconds)',
       'B_avg_LEG_landed', 'B_current_win_streak', 'R_avg_DISTANCE_landed',
       'B_Stance_Orthodox', 'R_avg_TD_pct', 'B_avg_SIG_STR_pct',
       'R_avg_opp_BODY_att'],
      dtype='object')

In [172]:
# Create a dataframe with only the top features
df_top_features = df[top_features]
df_test_id = df_test['id']
df_test = df_test[top_features]
df_test = pd.concat([df_test_id, df_test], axis=1)
df_top_features['B_Reach_cms'] = df['B_Reach_cms']
df_top_features.head()

C:\Users\USER\AppData\Local\Temp\ipykernel_26508\1332740388.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_top_features['B_Reach_cms'] = df['B_Reach_cms']


,B_Height_cms,B_Weight_lbs,B_age,B_total_rounds_fought,R_Reach_cms,weight_class,B_longest_win_streak,R_age,R_Height_cms,R_Weight_lbs,...,B_avg_opp_CTRL_time(seconds),B_avg_CTRL_time(seconds),B_avg_LEG_landed,B_current_win_streak,R_avg_DISTANCE_landed,B_Stance_Orthodox,R_avg_TD_pct,B_avg_SIG_STR_pct,R_avg_opp_BODY_att,B_Reach_cms
0,182.88,185.0,26.0,0.0,177.80,7,0.0,21.0,182.88,185.0,...,94.500000,99.750000,3.625000,0.0,18.250000,True,0.250000,0.450000,7.714127,190.50
1,177.80,155.0,27.0,19.0,187.96,5,3.0,30.0,180.34,170.0,...,14.601562,35.945312,5.492188,0.0,41.562500,True,0.554688,0.394141,17.593750,177.80
2,182.88,170.0,28.0,0.0,182.88,6,0.0,31.0,185.42,170.0,...,94.500000,99.750000,3.625000,0.0,18.250000,True,0.250000,0.450000,7.714127,190.50
3,175.26,185.0,28.0,17.0,185.42,6,4.0,34.0,177.80,170.0,...,67.156250,68.281250,2.664062,3.0,14.901989,False,0.419368,0.586479,8.554874,175.26
4,165.10,135.0,34.0,24.0,167.64,3,2.0,31.0,160.02,135.0,...,181.141357,209.931152,5.882080,0.0,33.382812,True,0.466719,0.381462,18.937500,170.18


In [173]:
df_top_features['B_Reach_cms'] = y

C:\Users\USER\AppData\Local\Temp\ipykernel_26508\2779498285.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_top_features['B_Reach_cms'] = y


In [174]:
# Export to CSV
df_top_features.to_csv('../Punya Andi/UFC_kombinasi7.csv', index=False)


In [175]:
df_test.to_csv('../regression_kaggle/UFC_kombinasi7.csv', index=False)